In [19]:
import os
import re

In [20]:
#폴더가 저장된 디렉토리 경로
folder_path1 = "./dataset" # 여기 경로 바꾸기

# model

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os

In [22]:
#이미지 전처리 
data_transforms = {
    'train':transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
    ]),
    'val' : transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
    ]),
}

In [23]:
# Define the data directory
data_dir= "dataset"

folder_path_list =os.listdir(folder_path1)


#print(os.listdir(folder_path)) ['0', '1', '10', '11', '12', '13', '14', '2', '3', '4', '5', '6', '7', '8', '9']
# Create data Loaders
#image_datasets = {x: datasets.ImageFolder(folder_path,data_transforms) for x in folder_path_list}
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir,x),data_transforms[x]) for x in ['train','val']}
#image_datasets


In [24]:
#dataloaders/
#dataloaders = {x:torch.utils.data.DataLoader(image_datasets[x],batch_size=4,shuffle=True,num_workers=0) for x in folder_path_list}
dataloaders = {x:torch.utils.data.DataLoader(image_datasets[x],batch_size=4,shuffle=True,num_workers=4) for x in ['train','val']}
#print(dataloaders)
dataset_sizes = {x: len(image_datasets[x]) for x in ['train','val']}
print(dataset_sizes)
#class_names = list(image_datasets.keys())
class_names = image_datasets['train'].classes
print(class_names)
#print(dataloaders['augmented'])

{'train': 20073, 'val': 704}
['0', '10', '102', '103', '104', '108', '11', '111', '112', '117', '119', '12', '121', '123', '127', '130', '131', '132', '138', '139', '14', '141', '144', '150', '151', '152', '155', '157', '158', '159', '162', '165', '166', '168', '169', '18', '181', '182', '183', '186', '19', '195', '198', '2', '20', '207', '212', '214', '215', '216', '217', '219', '22', '227', '228', '23', '230', '231', '235', '238', '24', '243', '245', '247', '249', '250', '252', '254', '255', '258', '26', '260', '262', '263', '269', '27', '271', '277', '279', '28', '280', '281', '3', '30', '34', '35', '41', '42', '45', '46', '49', '5', '50', '51', '52', '53', '54', '57', '58', '59']


In [25]:
# Load the pre-trained ResNet-18 model
model = models.resnet18(pretrained=True) 
# 학습된 resnet 18을 가져온다.
# Picking model from torch.Vision or torch.vision.model

# Freeze all layers except the final classification Layer
for name, param in model.named_parameters():
    
    if "fc" in name: # unfreeze the final classification layer
        # fully connect의 유무를 판단한다.
        
        param.requires_grad =True
        # will be trained
    else :
        param.requires_grad =False
        # freeze the layer
#Define the Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001,momentum=0.9)#SGD using stochastic gradient descent

# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device can use Gpu then use cuda, if not additionally using cpu
model = model.to(device)
#print(model)

#loss, optimizer, 활성화합수, softmax

## Training loop

In [15]:
#Training loop
loss_list=[]
acc_list=[]
num_epochs =50
for epoch in range(num_epochs):
    print(epoch)
    #for phase in class_names:
    for phase in ['train','val']:
        if phase =='train':#'augmented':
            model.train()
        else : 
            model.eval()
            
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs,1)
                loss = criterion(outputs, labels)
                
                if phase =='train':
                    loss.backward()
                    optimizer.step()
                    
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            
        epoch_loss = running_loss/ dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        acc_list.append(epoch_acc)
        loss_list.append(epoch_loss)
print("Training complete")

0
train Loss: 5.0189 Acc: 0.0433
val Loss: 7.5487 Acc: 0.1108
1
train Loss: 4.2331 Acc: 0.1080
val Loss: 7.2514 Acc: 0.1705
2
train Loss: 4.0027 Acc: 0.1418
val Loss: 7.1606 Acc: 0.1989
3
train Loss: 3.8871 Acc: 0.1656
val Loss: 7.2712 Acc: 0.2202
4
train Loss: 3.7901 Acc: 0.1819
val Loss: 7.0781 Acc: 0.2457
5
train Loss: 3.7569 Acc: 0.1900
val Loss: 7.3410 Acc: 0.2188
6
train Loss: 3.7212 Acc: 0.1962
val Loss: 7.1419 Acc: 0.2656
7
train Loss: 3.6728 Acc: 0.2042
val Loss: 7.6161 Acc: 0.2287
8
train Loss: 3.6606 Acc: 0.2056
val Loss: 7.7079 Acc: 0.2372
9
train Loss: 3.6478 Acc: 0.2122
val Loss: 7.7740 Acc: 0.2472
10
train Loss: 3.6441 Acc: 0.2116
val Loss: 7.6362 Acc: 0.2472
11
train Loss: 3.6277 Acc: 0.2138
val Loss: 7.7098 Acc: 0.2514
12
train Loss: 3.6250 Acc: 0.2144
val Loss: 7.5579 Acc: 0.2756
13
train Loss: 3.6095 Acc: 0.2165
val Loss: 7.6309 Acc: 0.2486
14
train Loss: 3.6042 Acc: 0.2202
val Loss: 7.7302 Acc: 0.2741
15
train Loss: 3.5831 Acc: 0.2199
val Loss: 7.8549 Acc: 0.2756
16

KeyboardInterrupt: 